In [22]:
from typing import List
import pandas as pd
import json
import os

def compute_validation_effectiveness(baseline_file: str, model_names: List, result_dir: str, index_name: str):

    df_baseline = pd.read_csv(baseline_file)

    #dup_columns = ["option_value", "option_name", "option_file", "dependent_option_name", "dependent_option_value", "dependent_option_file"]
    #duplicates = df_baseline[df_baseline.duplicated(subset=dup_columns)].index.to_list()
    #df_baseline = df_baseline.drop_duplicates(subset=dup_columns, keep="first")
    
    results = []

    for model_name in model_names:
        
        results_file = f"{result_dir}/all_dependencies_{index_name}_{model_name}.json"

        with open(results_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        true_positives = []
        true_negatives = []
        false_positives = []
        false_negatives = []
        accuracy_count = []
        skipped = 0

        for entry, (index, row) in zip(data, df_baseline.iterrows()):

            #if index in duplicates:
            #    continue
            
            response_dict = None
            isDependency = None

            rating = row["final_rating"]
            try:
                if "responses" in entry:
                    response = entry["responses"][0]
                else:
                    response = entry["response"]
                response_dict = json.loads(response, strict=False)
                isDependency = response_dict["isDependency"]
            except (json.JSONDecodeError, KeyError):
                skipped += 1
                continue
                
            if str(rating) == "Borderline":
                if isDependency:
                    accuracy_count.append(1)
                    true_positives.append(1)
                if not isDependency:
                    accuracy_count.append(1)
                    true_negatives.append(1)

            # TP: The LLM validates a dependency as correct and the dependency is correct
            if isDependency and str(rating).lower() == "true":
                accuracy_count.append(1)
                true_positives.append(1)
                
            # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
            if isDependency and str(rating).lower() == "false":
                accuracy_count.append(0)
                false_positives.append(1)

            # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
            if not isDependency and str(rating).lower() == "false":
                accuracy_count.append(1)
                true_negatives.append(1)

            # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
            if not isDependency and  str(rating).lower() == "true":
                accuracy_count.append(0)
                false_negatives.append(1)

        
        tp = sum(true_positives)
        fp = sum(false_positives)
        fn = sum(false_negatives)
        tn = sum(true_negatives)
        accuracy = sum(accuracy_count)/len(accuracy_count)


        print("Model: ", model_name)
        print("File: ", results_file)
        print("TP", tp)
        print("FP", fp)
        print("TN", tn)
        print("FN", fn)
        print("Sum correct response format: ", sum([tp, fp, tn, fn]))
        print("Entries skipped: ", skipped)

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score: ", f1_score)

        results.append({
            "model_name": model_name,
            "file": results_file,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1_score
        })
    

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{result_dir}/validation_effectiveness.csv", index=False)


compute_validation_effectiveness(
    baseline_file="../data/results/all_dependencies_updated.csv",
    model_names=["gpt-4o-2024-05-13", "gpt-3.5-turbo-0125", "llama3:70b", "llama3:8b"], #"llama3.1:70b"
    result_dir="../data/results/config4",
    index_name="all"
)

Model:  gpt-4o-2024-05-13
File:  ../data/results/config4/all_dependencies_all_gpt-4o-2024-05-13.json
TP 112
FP 29
TN 272
FN 87
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.768
Precision 0.7943262411347518
Recall 0.5628140703517588
F1 Score:  0.6588235294117647
Model:  gpt-3.5-turbo-0125
File:  ../data/results/config4/all_dependencies_all_gpt-3.5-turbo-0125.json
TP 143
FP 132
TN 169
FN 56
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.624
Precision 0.52
Recall 0.7185929648241206
F1 Score:  0.6033755274261603
Model:  llama3:70b
File:  ../data/results/config4/all_dependencies_all_llama3:70b.json
TP 145
FP 112
TN 189
FN 54
Sum correct response format:  500
Entries skipped:  0
Accuracy 0.668
Precision 0.5642023346303502
Recall 0.7286432160804021
F1 Score:  0.6359649122807017
Model:  llama3:8b
File:  ../data/results/config4/all_dependencies_all_llama3:8b.json
TP 132
FP 143
TN 96
FN 27
Sum correct response format:  398
Entries skipped:  102
Accuracy 0.572

In [24]:
import pandas as pd


file_path = "../data/results/without/validation_effectiveness.csv"

df = pd.read_csv(file_path)

df

,model_name,file,accuracy,precision,recall,f1-score
0,gpt-4o-2024-05-13,../data/results/without/all_dependencies_witho...,0.81800,0.891304,0.618090,0.729970
1,gpt-3.5-turbo-0125,../data/results/without/all_dependencies_witho...,0.68600,0.593750,0.668342,0.628842
2,llama3:70b,../data/results/without/all_dependencies_witho...,0.69200,0.627119,0.557789,0.590426
3,llama3:8b,../data/results/without/all_dependencies_witho...,0.66242,0.545455,0.843243,0.662420
